In [1]:
import numpy as np
#from math import comb (deprecated)
from scipy.special import comb
from time import time
from tqdm.notebook import tqdm, trange

# A class to allow operations on probability distributions, and protocol related classes

In [2]:
class Distribution: #numpy
#     A class to manage discret distributions
#     Contains a support list (self.n)
#     and a probability list (self.p) 
#     giving the probability we draw the corresponding value from the support
    def __init__(self, supports, probas):
        self.n = np.array(supports)
        self.p = np.array(probas)
        
    def order(self):
#         to sort the support, keeping the corresponding probability 
        order = np.argsort(self.n)
        self.n = self.n[order]
        self.p = self.p[order]
    
    def fuse(self):
#         if a support is sorted, and has repeated values, this fuses them with their probability
        supp = [self.n[0]]
        prob = [self.p[0]]
        for n,p in zip(self.n[1:],self.p[1:]):
            if n == supp[-1]:
                prob[-1] += p
            elif p>0 : 
                supp.append(n)
                prob.append(p)
        self.n = np.array(supp)
        self.p = np.array(prob)
        
    def normalize(self):
#         some manipulations (like filtering) lose normalization, this brings it back
        self.p = self.p/self.p.sum()
    
    def order_n_fuse(self):
#         applies the corresponding methods
        self.order()
        self.fuse()
    
    def combine(a,b):
        support = np.concatenate((a.n,b.n))
        probas = np.concatenate((a.p,b.p))
        res = Distribution(support, probas)
        res.order_n_fuse()
        return res
    
    def filtering(self, cond, norm = False):
#         filters over a condition (then normalize)
        ind = cond(self.n)
        self.p = self.p[ind]
        self.n = self.n[ind]
        if norm :
            self.normalize()
    
    def sum_cond(self,cond):
#         gives the probability we are in a set defined by the condition cond
        return self.p[cond(self.n)].sum()
        
    def function(a,f,b, sup_filter=None, normalize = False):
#         returns the probability distribution of f(a,b), 
#         we can force a condition sup_filter(a,b) over the supports of a and b 
        if type(b) == Distribution :
            val = f(a.n[:,None],b.n)
            prob = a.p[:,None] * b.p
            ind = np.full(val.shape,True)
            if sup_filter is not None :
                ind = sup_filter(val)
            distrib = Distribution(val[ind], prob[ind]) 
            distrib.order_n_fuse()            
            if sup_filter is not None and normalize:
                distrib.normalize()
            return distrib
        distrib = Distribution(f(a.n,b), a.p)
        if sup_filter is not None :
            distrib.filtering(sup_filter, normalize)
        return distrib
            
    def __add__(a,b):
#         returns the probability distribution of a+b, allows to actually use the a+b syntax
        return a.function(lambda x, y: x+y, b)
    
    def __mul__(a,b):
#         returns the probability distribution of a*b, allows to actually use the a*b syntax
        return a.function(lambda x, y: x*y, b)

    def __truediv__(a,b):
#         returns the probability distribution of a/b, allows to actually use the a/b syntax
#         WARNING: no control for /0
        return a.function(lambda x, y: x/y, b)

    def __neg__(a):
#         returns the probability distribution of -b, allows to actually use the -b syntax
        distrib = Distribution(-a.n[::-1], a.p[::-1]) 
        return distrib
    
    def __sub__(a,b):
#         returns the probability distribution of a-b, allows to actually use the a-b syntax
        return a.function(lambda x, y: x-y, b)

    def el_wise_add(a,b):
        return a.combine(b)
    
    def el_wise_sub(a,b):
        res  = a.combine(Distribution(b.n,-b.p))
        n_neg = res.n[np.where(res.p<0)]
        p_a = a.p[np.where(a == n_neg[:,None])[1]]
        p_b = b.p[np.where(b == n_neg[:,None])[1]]
        if (((p_b-p_a)/p_b) > 10**15).any() :
            print('warning: in el_wise_sub we get big negative probabilities, we put them to 0')
        res.p[np.where(res.p<0)] = 0
        return res
    
    def cut_geometric(N,p):
#         To create distribution that correspond to a truncated geometric distribution 
#         This corresponds to the probability distribution of the best priority you get at any level
        support = np.arange(N+1)
        probs = p * np.ones(N+1) * np.power(1-p,support)
        probs[-1] = (1-p)**N
        distrib = Distribution(support, probs)
        distrib.normalize()
        return distrib
    
    def binomial(N,p):
#         To create distribution that correspond to a binomial distribution 
#         This corresponds to the probability distribution of the number of endorsement we get at a level 
        supports = np.arange(N+1)
        probs = [comb(N,n) * s**n * (1-s)**(N-n) for n in supports]
        distrib = Distribution(supports, probs)
        distrib.normalize()
        return distrib
        
    def __str__(self):
        s = ''
        for n, p in zip(self.n, self.p):
            if type(n) == int :
                s = s + f'{n:10d} : {p:.2e}\n'
            else :
                s = s + f'{n:.2e} : {p:.2e}\n'
        return s[:-1]

note: probably want to upgrade the distribution class's fields to numpy arrays or torch tensors for speed

In [3]:
# very basic class for representing delay functions, 
# and sub class for emmy family
class Delay:
    def __init__(self, f):
        self.f = f
        
    def __call__(self, prio, endo):
        return self.f(prio, endo)

        
class Delay_emmy(Delay):
    def __init__(self, emmy):
        
        if emmy=='emmy':
            self.E = 32
            super().__init__( lambda p,e: 60 + 75*p)
        elif emmy=='emmy+':
            self.E = 32
            super().__init__( lambda p,e: 60 + 40*p + 8*np.maximum(self.E*3/4-e,0))
        elif emmy=='emmy*':
            self.E = 256
            def f(p,e):
                return 30 + ((p!=0) | (e<3*self.E/5)) * (30 + 40*p + 8*np.maximum(self.E*3/4-e,0))
#                 if p==0 and e>=3*self.E/5:
#                     return 30
#                 else:
#                     return 60 + 40*p + 8*np.maximum(self.E*3/4-e,0)
                    
            super().__init__( f)

# Test
To make sure things work  (to be skipped)

In [ ]:
delay = Delay_emmy('emmy+')
E = 32
P = 32
s = 0.2 # stake of attacker

# distribution of the attacker's number of endorsement
endo = Distribution.binomial(E,s)
# distribution of the attacker's best priority
prio = Distribution.cut_geometric(P,s)
# distribution of the honest bakers' best priority 
prio_minus = Distribution.cut_geometric(P,1-s)

In [ ]:
# we join the two priority distributions into 1 where we get an integer different from 0:
# when negative: the attacker has the priority 0 up to the |value|-1, 
# when positive: the honest bakers have the priority 0 up to the value - 1, 
# so the best priority for the attacker is minus the value 
# and the best priority for honest bakers is the value   

support = np.concatenate((-prio_minus.n[::-1][:-1],prio.n[1:]))
probas = np.concatenate((prio_minus.p[:0:-1], prio.p[1:]))
prio_compil = Distribution(support, probas)

# the functions to generate the two Delta distribution detailed in the overleaf 
delta_0_func = lambda p_c, e: delay(np.maximum(0,-p_c), E-e)-delay(np.maximum(0,p_c), E)
delta_l_func = lambda p_c, e: delay(np.maximum(0,-p_c), E-e)-delay(np.maximum(0,p_c), e)
delta_0 = prio_compil.function(delta_0_func,endo)
delta_l = prio_compil.function(delta_l_func,endo)

# the list for the probabilities of attacks of length l
F_l = delta_0
F = [None, F_l.sum_cond(lambda x:x>=0)]

F

each execution of the next cell will append a new value to F_l, so we get the probabilities for longer attacks,   
WARNING: takes much more time with each execution

In [ ]:
F_l = F_l + delta_l
F.append(F_l.sum_cond(lambda x:x>=0))
F

# Results

## Basic and Adjusted

### Basic

In [4]:
# Computations
P = 32
L = 4
for protocol, E in zip(["emmy+", "emmy*"], [32, 256]):
    for s in  [0.1,0.2,0.3,0.4]:
        delay = Delay_emmy(protocol)
        # distribution of the attacker's number of endorsement
        endo = Distribution.binomial(E,s)
        # distribution of the attacker's best priority
        prio = Distribution.cut_geometric(P,s)
        # distribution of the honest bakers' best priority 
        prio_minus = Distribution.cut_geometric(P,1-s)
        
        # we join the two priority distributions into 1 where we get an integer different from 0
        # when negative: the attacker has the priority 0 up to the |value|-1, 
        # when positive: the honest bakers have the priority 0 up to the value - 1, 
        # so the best priority for the attacker is minus the value 
        # and the best priority for honest bakers is the value   
        support = np.concatenate((-prio_minus.n[::-1][:-1],prio.n[1:]))
        probas = np.concatenate((prio_minus.p[:0:-1], prio.p[1:]))
        prio_compil = Distribution(support, probas)

        # the functions to generate the two Delta distribution detailed in the overleaf 
        delta_0_func = lambda p_c, e: delay(np.maximum(0,-p_c), E-e)-delay(np.maximum(0,p_c), E)
        delta_l_func = lambda p_c, e: delay(np.maximum(0,-p_c), E-e)-delay(np.maximum(0,p_c), e)
        delta_0 = prio_compil.function(delta_0_func,endo)
        delta_l = prio_compil.function(delta_l_func,endo)

        # the list for the probabilities of attacks of length l
        F_l = delta_0
        F = [None, F_l.sum_cond(lambda x:x>=0)]
        
        
        print(f" results for protocol {protocol}, with stake s = {s}, attacks up to length {L}. E = {E}, P = {P}:")
        print(f"Length {1}: {s} + {(F[-1]-s):.2e};",end=' ')
        for l in range (1,L):
            F_l = F_l + delta_l
            F.append(F_l.sum_cond(lambda x:x>=0))
            print(f"Length {l+1}: {F[-1]:.2e};",end=' ')
        print('\n')

 results for protocol emmy+, with stake s = 0.1, attacks up to length 4. E = 32, P = 32:
Length 1: 0.1 + 4.96e-07; Length 2: 1.42e-04; Length 3: 8.64e-08; Length 4: 5.94e-11; 

 results for protocol emmy+, with stake s = 0.2, attacks up to length 4. E = 32, P = 32:
Length 1: 0.2 + 9.69e-04; Length 2: 7.79e-03; Length 3: 2.24e-04; Length 4: 6.90e-06; 

 results for protocol emmy+, with stake s = 0.3, attacks up to length 4. E = 32, P = 32:
Length 1: 0.3 + 2.81e-02; Length 2: 8.12e-02; Length 3: 1.80e-02; Length 4: 4.22e-03; 

 results for protocol emmy+, with stake s = 0.4, attacks up to length 4. E = 32, P = 32:
Length 1: 0.4 + 1.36e-01; Length 2: 3.24e-01; Length 3: 2.03e-01; Length 4: 1.32e-01; 

 results for protocol emmy*, with stake s = 0.1, attacks up to length 4. E = 256, P = 32:
Length 1: 0.1 + -2.78e-17; Length 2: 4.28e-31; Length 3: 6.93e-62; Length 4: 7.61e-93; 

 results for protocol emmy*, with stake s = 0.2, attacks up to length 4. E = 256, P = 32:
Length 1: 0.2 + 6.59e-1

Note that we have an numerical issue for emmy*, stake 0.1, length 1 attacks. This is because in the non adjusted version of the calculation, the 0.1 added to the attack probability creates numerical errors.

Luckily we can still compute this result with our adjusted feasability or computing directly the result with the following formula which is obtained by expliciting the criteria for length-1 attacks: $$ D(0,E-e) \le D(p_l^-,E)$$ in the case where $$p_l^->0$$ as otherwise the non-adjusted attack is *successful*

$$e_l^- \ge max(103,64 + 5p_l^-)$$

In [5]:
# Computations
P = 32
E = 256
s = 0.1
delay = Delay_emmy("emmy*")
endo = Distribution.binomial(E,s)
prio = Distribution.cut_geometric(P,s)

accumulator = 0
for pl in prio.n :
    a = endo.sum_cond(lambda x: x>= (0 if pl==0 else max(103,64+pl)))
    if pl !=0 :    
        accumulator += a * prio.p[pl]

print(f'results for protocol emmy*, with stake s = {s}, attacks up of length 1. E = {E}, P = {P}')
print('We directly calculate it to avoid numerical errors and get:')
print(f'{s}+{accumulator:.2e}')


results for protocol emmy*, with stake s = 0.1, attacks up of length 1. E = 256, P = 32
We directly calculate it to avoid numerical errors and get:
0.1+4.63e-37


### Adjusted


In [6]:
# Computations
P = 32
L = 4
for protocol, E in zip(["emmy+", "emmy*"], [32, 256]):
    for s in  [0.1,0.2,0.3,0.4]:
        
        delay = Delay_emmy(protocol)
        endo = Distribution.binomial(E,s)
        prio = Distribution.cut_geometric(P,s)
        prio_minus = Distribution.cut_geometric(P,1-s)

        support = np.concatenate((-prio_minus.n[::-1][:-1],prio.n[1:]))
        probas = np.concatenate((prio_minus.p[:0:-1], prio.p[1:]))
        prio_compil = Distribution(support, probas)

        delta_0_func = lambda p_c, e: delay(np.maximum(0,-p_c), E-e)-delay(np.maximum(0,p_c), E)
        delta_l_func = lambda p_c, e: delay(np.maximum(0,-p_c), E-e)-delay(np.maximum(0,p_c), e)
        delta_0 = prio_compil.function(delta_0_func,endo)
        delta_l = prio_compil.function(delta_l_func,endo)

        # the list for the probabilities of attacks of length l
        F_l = delta_0
        F = [None, F_l.sum_cond(lambda x:x>=0)]
        # With the condition

        prio_valid = Distribution.cut_geometric(P,s)
        prio_valid.filtering(lambda x:x!=0,False)
        FC_l = prio_valid.function(delta_0_func, endo)
        FC_l2 = prio_valid.function(delta_0_func, endo)
        delta_cond = prio_valid.function(delta_l_func, endo)
        FC = [None, FC_l.sum_cond(lambda x:x>=0)]

        print(f" results for protocol {protocol}, with stake s = {s}, attacks up to length {L}. E = {E}, P = {P}:")
        print(f"Length {1}:{(FC[-1]):.2e};",end=' ')
        for l in range (L-1):
            F_l = F_l + delta_l
            F.append(F_l.sum_cond(lambda x:x>=0))
            part1 = FC_l + delta_l
            part2 = F_l  + delta_cond
            part3 = FC_l + delta_cond
            FC_l = (part1).el_wise_add(part2).el_wise_sub(part3)
            FC.append(FC_l.sum_cond(lambda x:x>=0))
            print(f"Length {l+1}: {FC[-1]:.2e};",end=' ')
        print('\n')


 results for protocol emmy+, with stake s = 0.1, attacks up to length 4. E = 32, P = 32:
Length 1:4.96e-07; Length 1: 5.02e-07; Length 2: 3.66e-10; Length 3: 2.70e-13; 

 results for protocol emmy+, with stake s = 0.2, attacks up to length 4. E = 32, P = 32:
Length 1:9.69e-04; Length 1: 2.02e-04; Length 2: 7.11e-06; Length 3: 2.40e-07; 

 results for protocol emmy+, with stake s = 0.3, attacks up to length 4. E = 32, P = 32:
Length 1:2.81e-02; Length 1: 1.17e-02; Length 2: 3.24e-03; Length 3: 8.89e-04; 

 results for protocol emmy+, with stake s = 0.4, attacks up to length 4. E = 32, P = 32:
Length 1:1.36e-01; Length 1: 1.38e-01; Length 2: 1.15e-01; Length 3: 9.42e-02; 

 results for protocol emmy*, with stake s = 0.1, attacks up to length 4. E = 256, P = 32:
Length 1:2.45e-37; Length 1: 2.39e-36; Length 2: 7.85e-68; Length 3: 2.68e-99; 

 results for protocol emmy*, with stake s = 0.2, attacks up to length 4. E = 256, P = 32:
Length 1:6.61e-14; Length 1: 2.98e-20; Length 2: 3.03e-36; 

## With variance reduction

### Basic

In [7]:
# Computations
P = 2
L = 4
for protocol, E in zip(["emmy+", "emmy*"], [32, 256]):
    for s in  [0.1,0.2,0.3,0.4]:
        delay = Delay_emmy(protocol)
        # distribution of the attacker's number of endorsement
        endo = Distribution([0,1],[1-s*E%1,s*E%1]) + int(s*E)
        
        # we directly define the joint priority distribution
        # when negative: the attacker has the priority 0 up to the |value|-1, 
        # when positive: the honest bakers have the priority 0 up to the value - 1, 
        # so the best priority for the attacker is minus the value 
        # and the best priority for honest bakers is the value   
        prio_compil = Distribution([-1,1], [s,1-s])

        # the functions to generate the two Delta distribution detailed in the overleaf 
        delta_0_func = lambda p_c, e: delay(np.maximum(0,-p_c), E-e)-delay(np.maximum(0,p_c), E)
        delta_l_func = lambda p_c, e: delay(np.maximum(0,-p_c), E-e)-delay(np.maximum(0,p_c), e)
        delta_0 = prio_compil.function(delta_0_func,endo)
        delta_l = prio_compil.function(delta_l_func,endo)

        # the list for the probabilities of attacks of length l
        F_l = delta_0
        F = [None, F_l.sum_cond(lambda x:x>=0)]

        print(f" results for protocol {protocol}, with stake s = {s}, attacks up to length {L}. E = {E}, P = {P}:")
        print(f"Length {1}: {s} + {(F[-1]-s):.2e};",end=' ')
        for l in range (1,L):
            F_l = F_l + delta_l
            F.append(F_l.sum_cond(lambda x:x>=0))
            print(f"Length {l+1}: {F[-1]:.2e};",end=' ')
        print('\n')
        

 results for protocol emmy+, with stake s = 0.1, attacks up to length 4. E = 32, P = 2:
Length 1: 0.1 + 0.00e+00; Length 2: 0.00e+00; Length 3: 0.00e+00; Length 4: 0.00e+00; 

 results for protocol emmy+, with stake s = 0.2, attacks up to length 4. E = 32, P = 2:
Length 1: 0.2 + 0.00e+00; Length 2: 0.00e+00; Length 3: 0.00e+00; Length 4: 0.00e+00; 

 results for protocol emmy+, with stake s = 0.3, attacks up to length 4. E = 32, P = 2:
Length 1: 0.3 + 0.00e+00; Length 2: 3.24e-02; Length 3: 0.00e+00; Length 4: 0.00e+00; 

 results for protocol emmy+, with stake s = 0.4, attacks up to length 4. E = 32, P = 2:
Length 1: 0.4 + 4.80e-01; Length 2: 1.60e-01; Length 3: 6.40e-02; Length 4: 2.56e-02; 

 results for protocol emmy*, with stake s = 0.1, attacks up to length 4. E = 256, P = 2:
Length 1: 0.1 + 0.00e+00; Length 2: 0.00e+00; Length 3: 0.00e+00; Length 4: 0.00e+00; 

 results for protocol emmy*, with stake s = 0.2, attacks up to length 4. E = 256, P = 2:
Length 1: 0.2 + 0.00e+00; Leng

### Adjusted

In [8]:
# Computations
P = 32
L = 4
for protocol, E in zip(["emmy+", "emmy*"], [32, 256]):
    for s in  [0.1,0.2,0.3,0.4]:
        
        delay = Delay_emmy(protocol)
        
        endo = Distribution([0,1],[1-s*E%1,s*E%1]) + int(s*E)
        
        # we directly define the joint priority distribution
        # when negative: the attacker has the priority 0 up to the |value|-1, 
        # when positive: the honest bakers have the priority 0 up to the value - 1, 
        # so the best priority for the attacker is minus the value 
        # and the best priority for honest bakers is the value   
        prio_compil = Distribution([-1,1], [s,1-s])

        delta_0_func = lambda p_c, e: delay(np.maximum(0,-p_c), E-e)-delay(np.maximum(0,p_c), E)
        delta_l_func = lambda p_c, e: delay(np.maximum(0,-p_c), E-e)-delay(np.maximum(0,p_c), e)
        delta_0 = prio_compil.function(delta_0_func,endo)
        delta_l = prio_compil.function(delta_l_func,endo)

        # the list for the probabilities of attacks of length l
        F_l = delta_0
        F = [None, F_l.sum_cond(lambda x:x>=0)]
        # With the condition

        prio_valid = Distribution([0,1],[s,1-s])
        prio_valid.filtering(lambda x:x!=0,False)
        FC_l = prio_valid.function(delta_0_func, endo)
        FC_l2 = prio_valid.function(delta_0_func, endo)
        delta_cond = prio_valid.function(delta_l_func, endo)
        FC = [None, FC_l.sum_cond(lambda x:x>=0)]

        print(f" results for protocol {protocol}, with stake s = {s}, attacks up to length {L}. E = {E}, P = {P}:")
        print(f"Length {1}:{(FC[-1]):.2e};",end=' ')
        for l in range (L-1):
            F_l = F_l + delta_l
            F.append(F_l.sum_cond(lambda x:x>=0))
            part1 = FC_l + delta_l
            part2 = F_l  + delta_cond
            part3 = FC_l + delta_cond
            FC_l = (part1).el_wise_add(part2).el_wise_sub(part3)
            FC.append(FC_l.sum_cond(lambda x:x>=0))
            print(f"Length {l+1}: {FC[-1]:.2e};",end=' ')
        print('\n')


 results for protocol emmy+, with stake s = 0.1, attacks up to length 4. E = 32, P = 32:
Length 1:0.00e+00; Length 1: 0.00e+00; Length 2: 0.00e+00; Length 3: 0.00e+00; 

 results for protocol emmy+, with stake s = 0.2, attacks up to length 4. E = 32, P = 32:
Length 1:0.00e+00; Length 1: 0.00e+00; Length 2: 0.00e+00; Length 3: 0.00e+00; 

 results for protocol emmy+, with stake s = 0.3, attacks up to length 4. E = 32, P = 32:
Length 1:0.00e+00; Length 1: 0.00e+00; Length 2: 0.00e+00; Length 3: 0.00e+00; 

 results for protocol emmy+, with stake s = 0.4, attacks up to length 4. E = 32, P = 32:
Length 1:4.80e-01; Length 1: 0.00e+00; Length 2: 0.00e+00; Length 3: 0.00e+00; 

 results for protocol emmy*, with stake s = 0.1, attacks up to length 4. E = 256, P = 32:
Length 1:0.00e+00; Length 1: 0.00e+00; Length 2: 0.00e+00; Length 3: 0.00e+00; 

 results for protocol emmy*, with stake s = 0.2, attacks up to length 4. E = 256, P = 32:
Length 1:0.00e+00; Length 1: 0.00e+00; Length 2: 0.00e+00; 